In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Preprocessing & Visualization

In [ ]:
df=pd.read_csv('/kaggle/input/hr-analytics-job-shift/HR analytics.csv')

In [ ]:
df.head()

In [ ]:
df.education_level.unique()

In [ ]:
df['experience'].unique()

In [ ]:
df.city=df.city.str.replace('city_', '')
df.city=df.city.astype(int)
df.gender=df.gender.replace({'Male':1,'Female':0,'Other':3})
df['gender']=df['gender'].astype(float)
df['HR Analytics']=df['HR Analytics'].replace({'No relevent experience':0,'Has relevent experience':1})
df['University Grade']=df['University Grade'].replace({'Grade 1':1,'Grade 2':2,'Grade 3':3})
df['education_level']=df['education_level'].replace({'nan':0,'Primary School':1,'High School':2,'Graduate':3,'Masters':4,'Phd':5})
df['Has HR expereince']=df['Has HR expereince'].replace({'No':0,'Yes':1})
df['experience']=df['experience'].replace({'<1':0,'>20':21,'nan':0,'NaN':0})
df['experience']=df['experience'].astype(float)
df['Job changes']=df['Job changes'].replace({'never':0,'nan':0,'>4':5,'NaN':0})
df['Job changes']=df['Job changes'].astype(float)
df['company_size']=df['company_size'].replace({'nan':0,'<10':2,'1-10':1,'50-99':3,'100-500':4,'500-999':5,'1000-4999':6,'5000-9999':7,'10000+':8,'NaN':0})
df['Get HR Analytics Job? ']=df['Get HR Analytics Job? '].replace({'No': 0, 'Yes': 1})

In [ ]:
df.head()

In [ ]:
df=df.fillna(0)

In [ ]:
df=df.drop('Training hours', axis=1)
df=df.drop('Employee_1D',axis=1)

In [ ]:
df=df.rename(columns={'Get HR Analytics Job? ':"Target"})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid")
ax = sns.boxplot(x=df["Target"],y=df['education_level'])

In [ ]:
ax = sns.boxplot(x=df["gender"],y=df['education_level'])

In [ ]:
ax = sns.boxplot(x=df["Target"],y=df['experience'])

In [ ]:
ax = sns.boxplot(x=df["Target"],y=df['Job changes'])

In [ ]:
plt.figure(figsize=(12, 12))
plt.legend(fontsize=10)
plt.tick_params(labelsize=10)
ax=sns.scatterplot(x=df['city'],y=df['University Grade'],hue=df['gender'],size=df['education_level'],data=df,sizes=(1,200))
plt.xticks(rotation=90)
ax.legend(loc='upper left',bbox_to_anchor=(1,1))

In [ ]:
df.info()

In [ ]:
df=df.replace({'No': 0, 'Yes': 1})

# 2. Classify Model by Py Caret

In [ ]:
pip install pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
df

In [ ]:
exp1 = setup(df,target = 'Target',silent=True)

In [ ]:
compare_models()

In [ ]:
cb = create_model('catboost')

In [ ]:
tuned_cb= tune_model(cb)

In [ ]:
plot_model(tuned_cb)

In [ ]:
interpret_model(tuned_cb)

In [ ]:
cb_holdout_pred = predict_model(cb)

In [ ]:
predictions = predict_model(cb, data=df)
predictions

# 3. Clustering Model by PyCaret

In [ ]:
df1=df.drop('Target', axis=1)

In [ ]:
from pycaret.clustering import *
df_clust = setup(df1, normalize = True,session_id = 123,silent=True)

In [ ]:
kmeans = create_model('kmeans',num_clusters = 5 )
print(kmeans)

In [ ]:
kmean_results = assign_model(kmeans)
kmean_results.head()

In [ ]:
kmean_results.groupby('Cluster').mean()

In [ ]:
plot_model(kmeans)

In [ ]:
plot_model(kmeans, plot = 'elbow')

In [ ]:
plot_model(kmeans, plot = 'silhouette')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'gender')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'University Grade')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'education_level')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'experience')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'Job changes')

In [ ]:
plot_model(kmeans, plot = 'distribution', feature = 'company_size')

In [ ]:
df['Cluster']=kmean_results['Cluster']
df

In [ ]:
df.groupby(['Cluster','Target'])['Target'].count().plot.bar()